### ingest  private-gpt  解析

```python
#### components/ingest_helper.py
# IngestionHelper  加载files，-> [Document] filter-node-metadata
"""  
BaseNode (抽象基类)
 ├─ TextNode
 │    └─ Document   ← 你最常用/读写的入口（整份文件或一页/一段）
 ├─ ImageNode
 ├─ IndexNode
 └─ ...（其他模态/复合节点）

- BaseNode：所有“可被索引/检索”的最小通用接口。
- TextNode：带 text 的文本节点。
- Document：实质上就是一个“整份文档/页”的 TextNode（许多版本里 Document 直接继承 TextNode）。读文件的 reader 通常返回 list[Document] 作为初始输入。
-  IngestionHelper.transform_file_into_documents() 里做的事：把各种文件读成 List[Document]，再做 metadata 规范化与清洗。


- BaseNode：统一抽象（ID、metadata、关系、可被索引/检索/持久化）。
- Document：读入阶段的“整文/整页文本节点”，是最自然的入口；从它向下切块得到更多 TextNode。
- Node（TextNode 等）：最小检索单元，被向量化、被重排、被拼成上下文回给 LLM。

# BaseNode 是什么（共同的骨架）
任何节点（文本、图片、索引引用…）都具备这些“基因”——常见属性/行为（名字可能随版本有细微差别，但概念稳定）：
id_：节点的唯一 ID（一个 chunk 一条 ID）。
metadata: dict：任意键值对（如 file_name, page_label, source 等）。
relationships：前后文/层级联系（如 PREVIOUS/NEXT/PARENT/SOURCE 等关系，用于重建上下文或来源）。
hash：内容哈希，便于幂等/变更检测。
excluded_embed_metadata_keys：做向量化时不带入的 metadata（减少噪声）。
excluded_llm_metadata_keys：喂给 LLM 上下文时不带入的 metadata（避免 prompt 污染）。
（有的版本）ref_doc_id 或通过关系把节点映射回原始文档，用于 delete_ref_doc 之类操作。
作用：统一“索引/存储/检索”的数据接口——向量库只关心它的文本与 embedding、LLM 只关心它的文本与必要 metadata、文档层级/前后窗口用 relationships 还原。

# TextNode：带文本的通用节点
在 BaseNode 基础上，TextNode提供：
text: str：节点文本（chunk 的正文）。
（常见）start_char_idx/end_char_idx：文本在原文中的切片位置（用于追踪来源、展示命中高亮）。
其他用于格式化/模板的字段（不同版本略有差异）。
作用：凡是“可被切块后的文本单元”，基本就是 TextNode（或其子类）。


# Document：入口文档，也是文本节点
Document 是 TextNode 的一个“角色化”版本，语义是“一份原始文档（或 PDF 的一页、或 HTML 的一个段落）”。在多数 Reader 里，读取一个文件会返回若干个 Document（PDF 常常是“每页一个 Document”）。
为什么工程里用它当入口？
Reader 输出：reader.load_data(path) → list[Document]
携带原始语义：你可以在 metadata 放 file_name, page_label, source, mtime 等，后面切块时会自动继承。
支持删除/追踪：很多删除操作（delete_ref_doc）是按“文档维度”生效的：你把文档的 doc_id 记录到每个 chunk（node）的 ref_doc 关系里，后面就能通过 doc_id 一键删掉这个文档对应的所有节点与向量。

document.metadata["file_name"] = file_name
document.metadata["doc_id"] = document.doc_id
document.excluded_embed_metadata_keys = ["doc_id"]
document.excluded_llm_metadata_keys = ["file_name", "doc_id", "page_label"]
给 Document 附上来源；
用 doc_id 当“文档级别的主键”，供后续删除/查询；
设置 excluded_*：避免把无关 metadata 污染 embedding/LLM。

"""

# components/ingest_component.py
"""
interface:
# llama_index.core.schema
    - Document：文本文档节点（继承自 TextNode/Node）；reader 输出的基本单位；携带 text/metadata/doc_id。
    - BaseNode：所有节点的抽象基类；insert_nodes() 接口处理的对象类型。
    - TransformComponent：变换组件协议（如 splitter/embedding）；run_transformations 的元素类型。
# llama_index.core.ingestion
    - run_transformations(documents, transformations, ...)：按顺序执行一串变换（典型是 [node_parser/splitter, embedder]），将 Document 变成带向量的 BaseNode 列表。
# llama_index.core.indices
    - VectorStoreIndex：RAG 常用的索引类型；支持 from_documents()、insert(document)、insert_nodes(nodes)、delete_ref_doc(doc_id, ...)。
    - load_index_from_storage(storage_context, ...)：从持久化存储中加载索引（与 StorageContext 配合）。
# llama_index.core.indices.base
    - BaseIndex：索引抽象基类（这里主要用作类型注解）。
# llama_index.core.data_structs
    - IndexDict：某些索引的内部数据结构类型参数（用于 BaseIndex[IndexDict] 的类型标注）。
# llama_index.core.storage
    - StorageContext：封装 vector_store / docstore / index_store 的上下文；负责持久化、加载、协调三者。
# llama_index.core.embeddings.utils
    - EmbedType：对“可用作嵌入器”的类型别名（抽象成一个联合/协议），便于函数签名与类型提示。
工程内自有部分
# IngestionHelper：文件→Documents 的解析与元数据规范化。
# local_data_path：存储持久化位置。
# Settings：工程配置（决定 ingest_mode / workers 等）。





"""



# settings.py





```